In [16]:
#Import Python libraries
import pandas as pd
import numpy as np
import time
import datetime
    
#Load data
path = 'prep_msn_29_fuel_leak_signals_preprocessed.csv'
parse_dates = ['UTC_TIME']
df = pd.read_csv(path, sep=',', parse_dates=parse_dates)
df = df.sort_values(['id','UTC_TIME'])

df = df.drop(['FUEL_USED_2', 'FUEL_USED_3', 'FUEL_USED_4',
       'FW_GEO_ALTITUDE', 'VALUE_FOB', 'VALUE_FUEL_QTY_FT1',
       'VALUE_FUEL_QTY_FT2', 'VALUE_FUEL_QTY_FT3', 'VALUE_FUEL_QTY_FT4',
       'FLIGHT_PHASE_COUNT', 'FUEL_USED_1','FUEL_ERROR_DIFF', 'FUEL_USED_TOT_DIFF',
       'FUEL_DEPLETION_DIFF'], axis=1)

#Create a dataframe for flights without leaks
no_leaks = df
no_leaks['Leak'] = 0

#Split column 'id' into 'AircraftRegistrationNumber' (also called 'MSN') and 'FlightNumber'
no_leaks[['AircraftRegistrationNumber', 'FlightNumber']] = no_leaks['id'].str.split('_', 1, expand=True)

#Group dataframe to obtian one row per flight
no_leaks_grouped = no_leaks.groupby('id').agg({'AircraftRegistrationNumber': 'last'})

#Assign randomly aircraft registration numbers and arrival airports to create artificial data to show the functionalities of the Streamlit app
no_leaks_grouped['AircraftRegistrationNumber'] = np.random.choice(['F-RBAC', 'F-TROE', 'F-RUWE','F-EAPE','F-MIPH', 'F-NATT','F-PZZA'], size=len(no_leaks_grouped))
no_leaks_grouped['ArrivalAirport'] = np.random.choice(['Paris Orly', 'Madrid Barajas', 'Berlin Brandenburg','London Heathrow','Seattle–Tacoma', 'Sydney'], size=len(no_leaks_grouped))
no_leaks = no_leaks.drop('AircraftRegistrationNumber', axis=1)
no_leaks = pd.merge(no_leaks, no_leaks_grouped.reset_index(), on='id')
no_leaks['id'] = no_leaks['AircraftRegistrationNumber'] +'_' + no_leaks['FlightNumber']  

#Add additional information about the  flight (airline, type, region)
airline_information_no_leaks = {'AircraftRegistrationNumber': ['F-RBAC', 'F-TROE', 'F-RUWE','F-EAPE','F-MIPH', 'F-NATT','F-PZZA'],
         'Airline': ['Airline C', 'Airline A', 'Airline B', 'Airline B', 'Airline C','Airline C','Airline A'],
        'Type': ['A220', 'A320', 'A330', 'A350', 'A380', 'A220', 'A320'],
         'Region': ['AMERICA', 'APAC', 'EMEA', 'EMEA', 'AMERICA', 'AMERICA', 'APAC']}
airline_information_no_leaks = pd.DataFrame(airline_information_no_leaks)
no_leaks = pd.merge(no_leaks, airline_information_no_leaks, on='AircraftRegistrationNumber')

#Create a dataframe for flights with leaks
leaks = df
leaks['Leak'] = 1

#Simulate a leakage for each of these flights to show the functionalities of the Streamlit app
leaks['FUEL_DEPLETION_ADD'] = (leaks.groupby('id').cumcount() * 0.5)
leaks['FUEL_ERROR_ADD'] =  (leaks.groupby('id').cumcount() * 0.5)
leaks['FUEL_DEPLETION'] = leaks['FUEL_DEPLETION'] + leaks['FUEL_DEPLETION_ADD']
leaks['FUEL_ERROR'] =  leaks['FUEL_ERROR'] + leaks['FUEL_ERROR_ADD']
leaks = df.drop(['FUEL_DEPLETION_ADD', 'FUEL_ERROR_ADD'], axis=1)

#Split column 'id' into 'AircraftRegistrationNumber' (also called 'MSN') and 'FlightNumber'
leaks[['AircraftRegistrationNumber', 'FlightNumber']] = leaks['id'].str.split('_', 1, expand=True)

#Group dataframe to obtian one row per flight
leaks_grouped = leaks.groupby('id').agg({'AircraftRegistrationNumber': 'last'})

#Assign aircraft registration numbers and arrival airports to create artificial data to show the functionalities of the Streamlit app
leaks_grouped['AircraftRegistrationNumber'] = ['L-ALPC', 'L-BEPC', 'L-CLPP', 'L-DLPW', 'L-ELPU', 'L-FLPC', 'L-GLCL', 'L-HLPC', 'L-ILPC', 'L-JLPC', 'L-KLPL', 'L-LLPC', 'L-MLUC', 'L-NLPC', 'L-OLPC', 'L-PLPC', 'L-QLPC', 'L-RLPC', 'L-SLPC', 'L-TLPC', 'L-ULPC', 'L-VLPC', 'L-WLPC', 'L-XLPC']
leaks_grouped['ArrivalAirport'] = np.random.choice(['Paris Orly', 'Madrid Barajas', 'Berlin Brandenburg','London Heathrow','Seattle–Tacoma', 'Sydney'], size=len(leaks_grouped))
leaks = leaks.drop('AircraftRegistrationNumber', axis=1)
leaks = pd.merge(leaks, leaks_grouped.reset_index(), on='id')
leaks['id'] = leaks['AircraftRegistrationNumber'] +'_' + leaks['FlightNumber']  

#Add additional information about the  flight (airline, type, region)
airline_information_leaks = {'AircraftRegistrationNumber': ['L-ALPC', 'L-BEPC', 'L-CLPP', 'L-DLPW', 'L-ELPU', 'L-FLPC', 'L-GLCL', 'L-HLPC', 'L-ILPC', 'L-JLPC', 'L-KLPL', 'L-LLPC', 'L-MLUC', 'L-NLPC', 'L-OLPC', 'L-PLPC', 'L-QLPC', 'L-RLPC', 'L-SLPC', 'L-TLPC', 'L-ULPC', 'L-VLPC', 'L-WLPC', 'L-XLPC'],
         'Airline': ['Airline A', 'Airline A', 'Airline C', 'Airline B', 'Airline C', 'Airline A', 'Airline A', 'Airline B', 'Airline B', 'Airline C', 'Airline ', 'Airline A', 'Airline B', 'Airline B', 'Airline C', 'Airline A', 'Airline A', 'Airline B', 'Airline B', 'Airline C', 'Airline ', 'Airline A', 'Airline B', 'Airline B'],
         'Type': ['A220', 'A380', 'A220', 'A350', 'A220', 'A220', 'A320', 'A350', 'A350', 'A380','A380', 'A320', 'A330', 'A350', 'A380','A220', 'A380', 'A330', 'A350', 'A380', 'A380', 'A320', 'A330', 'A380'],
        'Region': ['APAC', 'APAC', 'AMERICA', 'EMEA', 'AMERICA','APAC', 'APAC', 'EMEA', 'EMEA', 'AMERICA', 'APAC', 'APAC', 'EMEA', 'EMEA', 'AMERICA', 'APAC', 'APAC', 'EMEA', 'EMEA', 'AMERICA', 'APAC', 'APAC', 'EMEA', 'EMEA']}
airline_information_leaks = pd.DataFrame(airline_information_leaks)
leaks = pd.merge(leaks, airline_information_leaks, on='AircraftRegistrationNumber')

#Combine both dataframes 'no_leaks' and 'leaks'
all_flights = pd.concat([no_leaks, leaks])

#Create a csv file
all_flights.to_csv('streamlit-data.csv', index=False)


C:\Users\catta\AppData\Local\Temp\ipykernel_9712\1809962775.py:24: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  no_leaks[['AircraftRegistrationNumber', 'FlightNumber']] = no_leaks['id'].str.split('_', 1, expand=True)
C:\Users\catta\AppData\Local\Temp\ipykernel_9712\1809962775.py:56: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  leaks[['AircraftRegistrationNumber', 'FlightNumber']] = leaks['id'].str.split('_', 1, expand=True)


In [14]:
all_flights[(all_flights['Type'] == 'A220')&(all_flights['Region'] == 'AMERICA')].head()

UTC_TIME             id  duration  FUEL_DEPLETION  \
292544 2017-01-16 10:27:08  F-RBAC_2140.0       1.0             0.0   
292545 2017-01-16 10:27:09  F-RBAC_2140.0       2.0             4.0   
292546 2017-01-16 10:27:10  F-RBAC_2140.0       3.0             6.0   
292547 2017-01-16 10:27:11  F-RBAC_2140.0       4.0            10.0   
292548 2017-01-16 10:27:12  F-RBAC_2140.0       5.0            12.0   

        FUEL_USED_TOT  FUEL_ERROR  Leak FlightNumber  \
292544       1.132903   -1.132903     0       2140.0   
292545       2.265805    1.734195     0       2140.0   
292546       3.398708    2.601292     0       2140.0   
292547       4.531611    5.468389     0       2140.0   
292548       5.664513    6.335487     0       2140.0   

       AircraftRegistrationNumber   ArrivalAirport    Airline  Type   Region  
292544                     F-RBAC  London Heathrow  Airline C  A220  AMERICA  
292545                     F-RBAC  London Heathrow  Airline C  A220  AMERICA  
292546                     F-RBAC  London Heathrow  Airline C  A220  AMERICA  
292547                     F-RBAC  London Heathrow  Airline C  A220  AMERICA  
292548                     F-RBAC  London Heathrow  Airline C  A220  AMERICA

In [5]:
airline_information_leaks

AircraftRegistrationNumber    Airline  Type   Region
0                      L-ALPC  Airline C  A220  AMERICA
1                      L-BEPC  Airline A  A380     APAC
2                      L-CLPP  Airline B  A330     EMEA
3                      L-DLPW  Airline B  A350     EMEA
4                      L-ELPU  Airline C  A220  AMERICA
5                      L-FLPC   Airline   A220     APAC
6                      L-GLCL  Airline A  A320     APAC
7                      L-HLPC  Airline B  A350     EMEA
8                      L-ILPC  Airline B  A350     EMEA
9                      L-JLPC  Airline C  A380  AMERICA
10                     L-KLPL   Airline   A380     APAC
11                     L-LLPC  Airline A  A320     APAC
12                     L-MLUC  Airline B  A330     EMEA
13                     L-NLPC  Airline B  A350     EMEA
14                     L-OLPC  Airline C  A380  AMERICA
15                     L-PLPC   Airline   A220     APAC
16                     L-QLPC  Airline A  A380     APAC
17                     L-RLPC  Airline B  A330     EMEA
18                     L-SLPC  Airline B  A350     EMEA
19                     L-TLPC  Airline C  A380  AMERICA
20                     L-ULPC   Airline   A380     APAC
21                     L-VLPC  Airline A  A320     APAC
22                     L-WLPC  Airline B  A330     EMEA
23                     L-XLPC  Airline B  A380     EMEA

UTC_TIME             id  duration  FUEL_DEPLETION  \
0      2016-10-17 08:09:20  L-ALPC_2086.0       1.0             0.0   
1      2016-10-17 08:09:21  L-ALPC_2086.0       2.0             5.5   
2      2016-10-17 08:09:22  L-ALPC_2086.0       3.0             8.0   
3      2016-10-17 08:09:23  L-ALPC_2086.0       4.0             8.5   
4      2016-10-17 08:09:24  L-ALPC_2086.0       5.0             7.0   
...                    ...            ...       ...             ...   
331076 2017-06-29 08:41:58  L-XLPC_2223.0    2765.0          4633.0   
331077 2017-06-29 08:41:59  L-XLPC_2223.0    2766.0          4633.5   
331078 2017-06-29 08:42:00  L-XLPC_2223.0    2767.0          4634.0   
331079 2017-06-29 08:42:01  L-XLPC_2223.0    2768.0          4634.5   
331080 2017-06-29 08:42:02  L-XLPC_2223.0    2769.0          4635.0   

        FUEL_USED_TOT  FUEL_ERROR  Leak FlightNumber  \
0            1.779798   -1.779798     1       2086.0   
1            3.559596    1.940404     1       2086.0   
2            5.339394    2.660606     1       2086.0   
3            7.119193    1.380807     1       2086.0   
4            8.898991   -1.898991     1       2086.0   
...               ...         ...   ...          ...   
331076    4596.594741   36.405259     1       2223.0   
331077    4598.257162   35.242838     1       2223.0   
331078    4599.919584   34.080416     1       2223.0   
331079    4601.582005   32.917995     1       2223.0   
331080    4603.244426   31.755574     1       2223.0   

       AircraftRegistrationNumber   ArrivalAirport    Airline  Type   Region  
0                          L-ALPC  London Heathrow  Airline C  A220  AMERICA  
1                          L-ALPC  London Heathrow  Airline C  A220  AMERICA  
2                          L-ALPC  London Heathrow  Airline C  A220  AMERICA  
3                          L-ALPC  London Heathrow  Airline C  A220  AMERICA  
4                          L-ALPC  London Heathrow  Airline C  A220  AMERICA  
...                           ...              ...        ...   ...      ...  
331076                     L-XLPC   Seattle–Tacoma  Airline B  A380     EMEA  
331077                     L-XLPC   Seattle–Tacoma  Airline B  A380     EMEA  
331078                     L-XLPC   Seattle–Tacoma  Airline B  A380     EMEA  
331079                     L-XLPC   Seattle–Tacoma  Airline B  A380     EMEA  
331080                     L-XLPC   Seattle–Tacoma  Airline B  A380     EMEA  

[331081 rows x 13 columns]